# Kaggle Submission Step by Step


** Environment Setup **
load with kaggle cli

pip install kaggle-cli




** Config kaggle cli **
kg config -g -u 'username' -p 'password' -c 'Competettion Name'


In [2]:
# import stuff 
%matplotlib inline
path="data/redux/"
import utils; reload(utils)
from utils import *


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [3]:
batch_size=64
valid_size=2000
sample_size=200


In [4]:
# % is for magic commands 

# ! in notebook for the bash commands

import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/redux'


In [5]:
# create a directory structure

%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown


/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux


In [7]:
# in case where you dont have the proper directory structure and also to havee
# a small data set called sample to work with and also valid data set


#get all jpeg image names from train directory

%cd $DATA_HOME_DIR/train
g=glob('*.jpg')

shuf=np.random.permutation(g)

for i in range (valid_size): os.rename(shuf[i],'../valid/'+shuf[i])

/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/train


In [9]:
from shutil import copyfile

In [10]:
# create sample data set and copy files to the new folder structure of sample

g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/' + shuf[i])
    

In [11]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/' + shuf[i])

In [12]:
# Keras needs class wise data in each directory
%cd $DATA_HOME_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/sample/train
/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/sample/valid
/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/valid
/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/train


In [13]:



# For TEST Data create single 'unknown' class
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/



/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux/test


# Modeling and Finetuning


In [14]:
%cd $DATA_HOME_DIR

path=DATA_HOME_DIR+ '/' # change here to test it with sample data or train data

train_path=DATA_HOME_DIR+'/train/'
valid_path=DATA_HOME_DIR+'/valid/'
test_path=DATA_HOME_DIR+'/test/'

results_path=DATA_HOME_DIR+'/results/'


/home/ubuntu/notebooks/deeplearningProjects/CatOrDog/data/redux


In [15]:
# VGG Model from the tutorial 1

vgg=Vgg16()

In [16]:
batch_size=64
no_of_epochs=5

In [17]:
batches=vgg.get_batches(train_path,batch_size=batch_size)
val_batches=vgg.get_batches(valid_path,batch_size=2*batch_size)


# now fine tune as we have only 2 classes at the end

# linear classifier/ logistic at the end layer

vgg.finetune(batches)

# now set initially learning reate to be 0.01 that needs to be changed once
# get some satifying results and need to slow down the rate

vgg.model.optimizer.lr=0.01








Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# now save weights at each step so that it is easy to use them next time instead of 
# runing the model again and again

new_weights_filename=None



